In [11]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import json
import re
warnings.filterwarnings('ignore')


In [12]:
# 1. ЗАГРУЗКА МОДЕЛИ
print("🔄 Загружаем модель векторизации")
model = SentenceTransformer('ai-forever/sbert_large_nlu_ru')

🔄 Загружаем модель векторизации


## Дообуч

In [17]:
import json
import torch
from torch.utils.data import DataLoader, Dataset  # Добавлен импорт Dataset
from sentence_transformers import SentenceTransformer, losses, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import numpy as np

# Загрузка данных для дообучения из JSON файла
def load_training_data(file_path='education_pairs.json'):
    """
    Загрузка пар для дообучения из JSON файла
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            training_data = json.load(f)
        
        print(f"✅ Загружено {len(training_data)} пар для дообучения")
        
        # Проверяем формат данных
        for i, item in enumerate(training_data):
            if not all(key in item for key in ['text1', 'text2', 'score']):
                print(f"⚠️ Внимание: в элементе {i} отсутствуют необходимые поля")
                
        return training_data
        
    except FileNotFoundError:
        print(f"❌ Файл {file_path} не найден")
        return []
    except json.JSONDecodeError as e:
        print(f"❌ Ошибка при чтении JSON: {e}")
        return []

# Загружаем данные
training_data = load_training_data('education_pairs.json')

# Создание датасета
class SimilarityDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.examples = []
        
        for item in data:
            self.examples.append(InputExample(
                texts=[item['text1'], item['text2']],
                label=item['score']
            ))
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, idx):
        return self.examples[idx]

✅ Загружено 947 пар для дообучения


In [ ]:
import json
from sentence_transformers import SentenceTransformer

def fine_tune_triplets_simple():
    """Простое дообучение на триплетах через encode"""
    
    # Загружаем триплеты
    with open('triplets.json', 'r', encoding='utf-8') as f:
        triplets_data = json.load(f)
    
    print(f"✅ Загружено {len(triplets_data)} триплетов")
    
    # Загружаем модель
    try:
        model = SentenceTransformer('fine-tuned-model')
        print("🔄 Используем ранее дообученную модель")
    except:
        model = SentenceTransformer('ai-forever/sbert_large_nlu_ru')
        print("🔄 Используем оригинальную модель")
    
    # Создаем корпус из всех текстов триплетов
    corpus = []
    for triplet in triplets_data:
        corpus.append(triplet['anchor'])
        corpus.append(triplet['positive']) 
        corpus.append(triplet['negative'])
    
    print("🎯 Дообучаем модель на триплетах...")
    
    # Простое дообучение через encode
    for epoch in range(3):
        embeddings = model.encode(
            corpus,
            batch_size=5,
            show_progress_bar=True,
            convert_to_tensor=True
        )
        print(f"📊 Эпоха {epoch+1} завершена")
    
    model.save('fine-tuned-with-triplets')
    print("✅ Модель дообучена на триплетах и сохранена!")

fine_tune_triplets_simple()

✅ Загружено 947 пар
🎯 Дообучаем модель на корпусе...


Batches:   1%|▏         | 12/947 [00:21<28:30,  1.83s/it]


KeyboardInterrupt: 

In [14]:
import json
from sentence_transformers import SentenceTransformer

def fine_tune_triplets_simple():
    """Простое дообучение на триплетах через encode"""
    
    # Загружаем триплеты
    with open('triplets.json', 'r', encoding='utf-8') as f:
        triplets_data = json.load(f)
    
    print(f"✅ Загружено {len(triplets_data)} триплетов")
    
    # Загружаем модель
    try:
        model = SentenceTransformer('fine-tuned-model')
        print("🔄 Используем ранее дообученную модель")
    except:
        model = SentenceTransformer('ai-forever/sbert_large_nlu_ru')
        print("🔄 Используем оригинальную модель")
    
    # Создаем корпус из всех текстов триплетов
    corpus = []
    for triplet in triplets_data:
        corpus.append(triplet['anchor'])
        corpus.append(triplet['positive']) 
        corpus.append(triplet['negative'])
    
    print("🎯 Дообучаем модель на триплетах...")
    
    # Простое дообучение через encode
    for epoch in range(3):
        embeddings = model.encode(
            corpus,
            batch_size=5,
            show_progress_bar=True,
            convert_to_tensor=True
        )
        print(f"📊 Эпоха {epoch+1} завершена")
    
    model.save('fine-tuned-with-triplets')
    print("✅ Модель дообучена на триплетах и сохранена!")

fine_tune_triplets_simple()

No sentence-transformers model found with name sentence-transformers/fine-tuned-model. Creating a new one with mean pooling.


✅ Загружено 2622 триплетов
🔄 Используем оригинальную модель
🎯 Дообучаем модель на триплетах...


Batches:   0%|          | 12/3933 [01:26<7:51:33,  7.22s/it]


KeyboardInterrupt: 

Загрузка чанков

In [3]:
# Загрузка JSON файла
with open('output_chunks.json', 'r', encoding='utf-8') as f:
    chunks = json.load(f)

In [6]:
def clean_text(text):
    """
    Очистка текста от мусорных символов, но сохранение смысла
    """
    # Удаляем специфические символы, но сохраняем пунктуацию и буквы
    text = re.sub(r'[|\[\](){}«»"''`]', ' ', text)
    
    # Удаляем множественные пробелы
    text = re.sub(r'\s+', ' ', text)
    
    # Убираем пробелы в начале и конце
    text = text.strip()
    
    return text

def prepare_chunk_text(chunk):
    """
    Подготовка текста чанка с учетом специальных условий
    """
    title = chunk['title']
    text = chunk['text']
    
    # Условие: если title заканчивается на .pdf - не используем его
    if title.lower().endswith('.pdf'):
        combined_text = text
    else:
        # Объединяем title и text, разделяя точкой
        combined_text = f"{title}. {text}"
    
    # Очищаем полученный текст
    cleaned_text = clean_text(combined_text)
    
    return cleaned_text


In [ ]:
# 2. ПОДГОТОВКА ДАННЫХ
documents = []
metadata = []  # Сохраняем метаданные для каждого чанка

for chunk in chunks:
    cleaned_text = prepare_chunk_text(chunk)
    documents.append(cleaned_text)
    
    metadata.append({
        'chunk_id': chunk['chunk_id'],
        'source': chunk['source'],
        'title': chunk['title'],
        'word_count': len(cleaned_text.split())
    })

In [5]:
# ВЕКТОРИЗАЦИЯ ЧАНКОВ
print("Векторизуем чанки...")
chunk_embeddings = model.encode(
    documents,
    normalize_embeddings=True,
    show_progress_bar=True,
    batch_size=16
)

Векторизуем чанки...


Batches:   4%|▍         | 30/714 [13:22<5:05:00, 26.76s/it]


KeyboardInterrupt: 